In [2]:
!pip install yfinance
!pip install pandas_datareader

  Preparing metadata (setup.py) ... done
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8488 sha256=1ab3480b8031f991284ff710ad8d3be664e1e99950f2e0e0be399db0a094e23c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/15/e6/fa/f4bf8d84e804547b3c1b1d4b09a671768502b32ca33ec60651
Successfully built multitasking


In [63]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data

In [77]:
df_tickers = pd.read_csv('Ticker List.csv')
tickers = list(df_tickers['Symbol'])

start_dt = '2020-01-01'
end_dt = '2021-12-30'

# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
interval_ = '1d'

# select period high/low
period_ = 52

# get marketcap info for all tickers
try:
    mkt_cap = data.get_quote_yahoo(tickers)['marketCap']
except (KeyError):
    pass

In [65]:
def getTickerData(tickers,start_dt,end_dt,interval_,period_):
    output = []
    for ticker in tickers:
        data = yf.download(ticker, start=start_dt, end=end_st, interval=interval_)
        data.reset_index(inplace=True)
        
        data['Ticker'] = ticker
        data['Period'] = period_
        data['P_High'] = data['High'].rolling(period_).max()
        data['P_Low'] = data['Low'].rolling(period_).min()
        data['P_Spread'] = data['P_High'] - data['P_Low']
        data['P_Percentile'] = (data['Close'] - data['P_Low'])/data['P_Spread']
        data.dropna(inplace=True)
        
        output.append(data)
        
    df = pd.concat(output)
    return(df)
     

def getMarketCap():
    output = []
    for ticker in tickers:
        if ticker in ['INFO','KSU','VIAC','WLTW']:
            pass
        else:
            try:
                print("Working: ", ticker)
                output.append(data.get_quote_yahoo(ticker)['marketCap'])
            except(KeyError) as e1:
                print("ERROR: ", e1, ticker)
                pass
            except(RemoteDataError) as e2:
                print("ERROR: ", e2, ticker)
                pass
            except(NameError) as e3:
                print("ERROR: ", e3, ticker)
                pass
    df_mkt = pd.concat(output)
    return(df_mkt)        

In [158]:
df = getTickerData(tickers,start_dt,end_dt,interval_,period_)
keep_cols = ['Date','Ticker', 'P_Percentile']
df_stocks = df[keep_cols].copy()


df_mkt = getMarketCap()
df_mkt = pd.DataFrame(df_mkt).reset_index()
df_mkt.columns = ['Ticker','MarketCap']  
df_mkt['MarketCap_Percentile'] = df_mkt['MarketCap'] / df_mkt['MarketCap'].sum()

#merge 
df_summary = df_stocks.merge(df_mkt)
df_summary['MarketCap_Billions'] = df_summary['MarketCap']/1000000000

# create mktshr bins
cut_labels_4 = ['01. LT 5B', '02. Btwn 5-10B', '03. Btwn 10-20B', 
                '04. Btwn 20-40B', '05. Btwn 40-60B', '06. Btwn 60-250B',
               '07. Btwn 250-750B','08. Btwn 750-1T', '09. GT 1T']
cut_bins = [0, 5, 10, 20, 40, 60, 250, 750, 1000, 100000000]
df_summary['MarketCap_Billions_Bins'] = pd.cut(df_summary['MarketCap_Billions'], bins=cut_bins, labels=cut_labels_4)

# # create mktshr percentile bins
# cut_labels_4 = ['01. LT 5pct', '02. Btwn 5-10pct', '03. Btwn 10-20pct', 
#                 '04. Btwn 20-40pct', '05. Btwn 40-60pct', '06. Btwn 60-80pct',
#                '07. Btwn 80-90pct','08. Btwn 90-95pct', '09. GT 95pct']
# cut_bins = [0, 0.05, 0.10, 0.20, 0.40, 0.60, 0.80, 0.90, 0.95, 1.0]
# df_summary['P_MarketCap_Percentile_Bins'] = pd.cut(df_summary['MarketCap_Percentile'], bins=cut_bins, labels=cut_labels_4)


# create percentile bins
cut_labels_4 = ['01. LT 5pct', '02. Btwn 5-10pct', '03. Btwn 10-20pct', 
                '04. Btwn 20-40pct', '05. Btwn 40-60pct', '06. Btwn 60-80pct',
               '07. Btwn 80-90pct','08. Btwn 90-95pct', '09. GT 95pct']
cut_bins = [0, 0.05, 0.10, 0.20, 0.40, 0.60, 0.80, 0.90, 0.95, 1.0]
df_summary['P_Percentile_Bins'] = pd.cut(df_summary['P_Percentile'], bins=cut_bins, labels=cut_labels_4)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- INFO: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- VIAC: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********

Working:  PLD
Working:  PRU
Working:  PTC
Working:  PEG
Working:  PSA
Working:  PHM
Working:  PVH
Working:  QRVO
Working:  QCOM
Working:  PWR
Working:  DGX
Working:  RL
Working:  RJF
Working:  RTX
Working:  O
Working:  REG
Working:  REGN
Working:  RF
Working:  RSG
Working:  RMD
Working:  RHI
Working:  ROK
Working:  ROL
Working:  ROP
Working:  ROST
Working:  RCL
Working:  SPGI
Working:  CRM
Working:  SBAC
Working:  SLB
Working:  STX
Working:  SEE
Working:  SRE
Working:  NOW
Working:  SHW
Working:  SPG
Working:  SWKS
Working:  SNA
Working:  SO
Working:  LUV
Working:  SWK
Working:  SBUX
ERROR:  'marketCap' SBUX
Working:  STT
Working:  STE
Working:  SYK
Working:  SIVB
Working:  SYF
Working:  SNPS
Working:  SYY
Working:  TMUS
Working:  TROW
Working:  TTWO
Working:  TPR
Working:  TGT
Working:  TEL
Working:  TDY
Working:  TFX
Working:  TER
Working:  TSLA
Working:  TXN
Working:  TXT
Working:  COO
Working:  HIG
Working:  HSY
Working:  MOS
Working:  TRV
Working:  DIS
Working:  TMO
Working:  TJX


KeyError: 'MarketCap_Billions'

In [155]:
df_summary.to_csv('temp.csv')